### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
# importando a biblioteca ADS e realizando a autenticação
import ads
ads.set_auth("resource_principal")

In [ ]:
from ads.jobs import DataFlow, Job, DataFlowRuntime
from uuid import uuid4
import os

name = "Dataflow Batch Run"
dataflow_configs = (
    DataFlow()
    .with_compartment_id(os.environ["NB_SESSION_COMPARTMENT_OCID"])
    .with_logs_bucket_uri("oci://dssilva-metastore@idcci5ks1puo")
    .with_driver_shape("VM.Standard.E4.Flex")
    .with_driver_shape_config(ocpus=1, memory_in_gbs=8)
    .with_num_executors(4)
    .with_executor_shape("VM.Standard.E4.Flex")
    .with_executor_shape_config(ocpus=1, memory_in_gbs=8)
    .with_spark_version("3.2.1")
)
runtime_config = (
    DataFlowRuntime()
    .with_script_uri("read_tlc.py")
    .with_script_bucket("oci://dssilva-metastore@idcci5ks1puo")
#     .with_custom_conda("oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/PySpark_3.2_and_Data_Flow/3.0/pyspark32_p38_cpu_v3")
)
df = Job(name=name, infrastructure=dataflow_configs, runtime=runtime_config)
df.create()